# The Refinery Files 0x02: Amadey Loader

This is a short tutorial about decrypting the strings in the following Amadey Loader sample:
```
6e01f9d1997186d06274a508bc0a511aa6fb50e430b77efca593c00d3fc62cba
```
As always, remember that this is **malware**, do not execute it unless you know exactly what you are doing. For instructions about how to set up [refinery], see the main page and documentation.

[refinery]: https://github.com/binref/refinery/

In [1]:
from tutorials import boilerplate
boilerplate.store_sample(
    name='a.bin',
    hash='6e01f9d1997186d06274a508bc0a511aa6fb50e430b77efca593c00d3fc62cba'
)

In [1]:
%ls

00.223 MB 6e01f9d1997186d06274a508bc0a511aa6fb50e430b77efca593c00d3fc62cba a.bin


## String Decryption

After some reverse engineering, you discover that the strings in this binary can be identified as the ones that are in uppercase hex format. After hex-decoding them, they are decrypted by sequentially subtracting the bytes of a key string. This is the key string:
```
6768875d0dd576a718d85aa1d71d25c1
```
... but oddly enough, the malware's decryption routine adds `1` to every index when accessing this buffer. This means that it will never access the very first byte of the key, but it will also use the zero byte that terminates it. A quick way to extract the C2 servers from this sample would therefore be the following:

In [1]:
%emit a.bin | carve -dn10 hex [| sub q:768875d0dd576a718d85aa1d71d25c1%00 | xtp domain | defang | sep ]

happyday9risce[.]com
xksldjf9sksdjfks[.]com
dhisa8f9ah02hopasiaf[.]com


Here, we have first used [carve] using the `hex` pattern to extract strings that are likely encrypted. We used the `-n` switch to select a minimum length of 10 characters, which corresponds to 5 decoded bytes. We also used the `-d` (or `--decode`) flag to immediately hex-decode the matched strings. We then open a [frame] so that we can work on each of these strings individually. Each string is then decrypted by subtracting the bytes of a key using the [sub] unit; this key is generated from the following [multibin] expression:
```
q:768875d0dd576a718d85aa1d71d25c1%00
```
This represents the result of URL-decoding the string, which is the decryption key with the first letter removed and a zero byte appended; these two modifications account for the off-by-one access used in Amadey's code. After decrypting the string, we run [xtp] on it, which is short for "extract pattern": The unit is designed to extract indicators. There are a number of patterns available by name, in this case we are interested in domains. Run `xtp -h` to obtain a list of all currently available indicator patterns. Finally, we use [defang] because these domains will end up in a Jupiter notebook and we want to prevent that some web component somewhere becomes overeager and turns them into a clickable link. Because we would like to have the outputs separated by new lines in the terminal, we finally pipe the results to [sep] before closing the frame.

[carve]: https://binref.github.io/#refinery.carve
[defang]: https://binref.github.io/#refinery.defang
[frame]: https://binref.github.io/lib/frame.html
[multibin]: https://binref.github.io/lib/argformats.html
[sub]: https://binref.github.io/#refinery.sub
[sep]: https://binref.github.io/#refinery.sep
[xtp]: https://binref.github.io/#refinery.xtp

The pipeline is fine but just for the sake of keeping this tutorial going we can try to:
1. First extract potential keys based on a wild guess of the format,
2. then try to decrypt all the strings with all the key candidates and keep only the good stuff.

## Extracting Key Candidates

After a little more reverse engineering, you notice that there are actually three strings in the binary that look likey key material:

1. `6768875d0dd576a718d85aa1d71d25c1`
2. `396554bad854c42cee4903aadccae3d4`
3. `f12bb04fdd6d0132403f4b3bd4d4814b`

All of them are 32 characters long and use lowercase hex characters. Notably, the encrypted strings all used **uppercase** hex characters. There is reason to believe that the other two sequences are used as decryption keys for other kinds of data, but that's something you can figure out later. For now, we would like to devise a refinery pipeline that has a chance to work on other samples where the string encryption key might be different. We will, however, assume that the format of the key will always be lowercase hex and 32 characters in length. This is clearly not a safe assumption, but I'll need you to get all the way off of my back about that so that the tutorial can happen. First, let's get all the potential keys out of our sample:

In [1]:
%emit a.bin | rex [a-f0-9]{32} [| cfmt {index+1}. {} ]]

1. 6768875d0dd576a718d85aa1d71d25c1
2. 396554bad854c42cee4903aadccae3d4
3. f12bb04fdd6d0132403f4b3bd4d4814b


First, we use [rex] (the regular expression unit) to search for all 32-letter lowercase hex strings. For each of those, we invoke the [cfmt] unit so that I can talk about it in this tutorial. Its command line arguments are interpreted as a Python format string expression which can use any [meta] variables and the symbol `{}`, the latter representing the body of the current chunk. In this case, we use the magic [meta] variable `index` which represents the index of the current chunk in the frame. Since the chunk index is `0`-based, we have to add `1` in order to have the output list start at `1`. Finally, notice that we did not use the [sep] unit, but the output was separated by newlines anyway: This happened because we used `]]` rather than `]` to close the frame; using more closing brackets than necessary will insert newlines between all output chunks. This is an intentional piece of syntactic sugar which has been added for convenience.

Now we have a list of all prospect keys, but what we'd really want is to have three chunks, each of which contains the contents of `a.bin`, but each also carrying a different prospect key as a piece of [meta]data. We can achieve this like so:

[cfmt]: https://binref.github.io/#refinery.cfmt
[rex]: https://binref.github.io/#refinery.rex
[sep]: https://binref.github.io/#refinery.sep
[meta]: https://binref.github.io/lib/meta.html

In [1]:
%emit a.bin | rex [a-f0-9]{32} [| swap key | emit a.bin | peek -ml0 ]

------------------------------------------------------------------------------------------------------------------------
  entropy = 80.76%
      key = 6768875d0dd576a718d85aa1d71d25c1
    magic = PE32 executable (GUI) Intel 80386, for MS Windows
   offset = 0x2FBBC
     size = 00.223 MB
------------------------------------------------------------------------------------------------------------------------
  entropy = 80.76%
      key = 396554bad854c42cee4903aadccae3d4
    magic = PE32 executable (GUI) Intel 80386, for MS Windows
   offset = 0x2FBE0
     size = 00.223 MB
------------------------------------------------------------------------------------------------------------------------
  entropy = 80.76%
      key = f12bb04fdd6d0132403f4b3bd4d4814b
    magic = PE32 executable (GUI) Intel 80386, for MS Windows
   offset = 0x2FC0C
     size = 00.223 MB
------------------------------------------------------------------------------------------------------------------------


What we did here is to first [emit] the contents of `a.bin`. As before, we then extract all key candidates from it, opening a new [frame] to process each of these strings individually. However, we will want to work on the contents of `a.bin` again, not the key string itself. To get to a point where we can do that, we first use [swap] to move the contents of the current chunk into the [meta]-variable `key`. After invoking the [swap] unit, the frame contains three empty chunks, each of which has one of the key candidates attached to it as a variable. Then we simply run [emit] again which gives us three chunks, each of which contains the contents of `a.bin`.

[emit]: https://binref.github.io/#refinery.emit
[swap]: https://binref.github.io/#refinery.swap
[frame]: https://binref.github.io/lib/frame.html
[meta]: https://binref.github.io/lib/meta.html

## Now All Together

The final step is literally just sticking the two pipelines together:

In [1]:
%emit a.bin [| rex [a-f0-9]{32} | swap key | emit a.bin | carve -dn10 hex | sub cca[h:00]:snip[1:]:var:key | xtp | defang ]]

happyday9risce[.]com
xksldjf9sksdjfks[.]com
dhisa8f9ah02hopasiaf[.]com


Since the key is now stored as a variable, we need to do a little more work to account for the off-by-one glitch in Amadey's code. First, the expression `var:key` contains the contents of the `key` variable of the current chunk. Next, `snip[1:]:var:key` skips the first byte and finally, `cca[h:00]:take[1:]:var:key` appends a zero byte. This [multibin] expression uses both [snip] and [cca] as unit-based handlers.

[cca]: https://binref.github.io/#refinery.cca
[multibin]: https://binref.github.io/lib/argformats.html
[snip]: https://binref.github.io/#refinery.snip